# TensorFlow playground

## Simple TDNN

In [64]:
import tensorflow as tf
import numpy as np

In [65]:
x_c = np.array([[[3,0,0],[0,3,0],[0,0,3],[4,0,0],[0,4,0]]], dtype=float)
y_c = np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0]], dtype=float)

In [71]:
def multilayer_perceptron(x, weights):
#     layer_1 = tf.matmul(x, weights['h1'])
    layer_1 = tf.nn.pool(x,
        window_shape=[2],
        pooling_type='AVG',
        padding='VALID',
        name='Sliding_avg'
    )
    layer_1 = tf.nn.moments(layer_1,
        axes=[1],
        name='Stats_generating'
    )
    return layer_1

In [72]:
W = x_c.shape[1]
C = x_c.shape[2]
n_classes = y_c.shape[1]
N = x_c.shape[0]

weights = {
    'h1': tf.Variable(tf.ones([N, n_classes]))
}

In [73]:
training_epochs = 3
display_step = 1
batch_size = 3

x = tf.placeholder("float", [None, W, C])
y = tf.placeholder("float", [None, n_classes])

In [74]:
predictions = multilayer_perceptron(x, weights)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

tf.summary.histogram("predictions", predictions)

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Variable:0' shape=(1, 3) dtype=float32_ref>", "<tf.Variable 'Variable_1:0' shape=(1, 3) dtype=float32_ref>"] and loss Tensor("Mean:0", shape=(), dtype=float32).

In [ ]:
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter( './logs/train ', sess.graph)
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_c) / batch_size)
        x_batches = np.array_split(x_c, total_batch)
        y_batches = np.array_split(y_c, total_batch)
        for i in range(total_batch):
            counter += 1
            batch_x, batch_y = x_batches[i], y_batches[i]
            print(batch_x.shape, batch_y.shape)
            merge = tf.summary.merge_all()
            
#             summary, batch_loss, new_state, _ = sess.run([merge, model.loss, model.final_state, model.optimizer],feed_dict=feed)
            
            summary, c, _ = sess.run([merge, cost, optimizer], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y
                            })
            train_writer.add_summary(summary, counter)
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#     print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))    
    print("Accuracy:", accuracy.eval({x: x_c, y: y_c}))

In [63]:
tf.reset_default_graph()